<a href="https://colab.research.google.com/github/profteachkids/CHE3022_Fall2021/blob/main/2DHeatConductionCartesian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default='plotly_dark'
from scipy.optimize import newton_krylov

In [6]:
nx, ny = 50, 50
hx, hy = 1./(nx-1), 1./(ny-1)

T_left, T_right = 100., 200.
T_top, T_bottom = 250., 150.
qk = 1000.

def residual(T):
    d2x = np.zeros_like(T)
    d2y = np.zeros_like(T)

    d2x[1:-1] = (T[2:]   - 2*T[1:-1] + T[:-2]) / hx/hx
    d2x[0]    = (T[1]    - 2*T[0]    + T_left)/hx/hx
    d2x[-1]   = (T_right - 2*T[-1]   + T[-2])/hx/hx

    d2y[:,1:-1] = (T[:,2:] - 2*T[:,1:-1] + T[:,:-2])/hy/hy
    d2y[:,0]    = (T[:,1]  - 2*T[:,0]    + T_bottom)/hy/hy
    d2y[:,-1]   = (T_top   - 2*T[:,-1]   + T[:,-2])/hy/hy

    return d2x + d2y + qk

# solve
guess = np.zeros((nx, ny))
sol = newton_krylov(residual, guess, method='lgmres', verbose=0)

In [7]:
x, y = np.mgrid[0:1:(nx*1j), 0:1:(ny*1j)]
fig = make_subplots()
for i in np.arange(nx):
     fig.add_trace(go.Scatter3d(x=x[i,:], y=y[i,:], z=sol[i,:], mode='lines', line_color='gray'))

for j in np.arange(ny):
     fig.add_trace(go.Scatter3d(x=x[:,j], y=y[:,j], z=sol[:,j], mode='lines', line_color='gray'))

fig.add_trace(go.Surface(x=x,y=y,z=sol))

fig.update_layout(width=800, height=800, scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='T'),
                  showlegend=False)